In [1]:
import chess
import concurrent.futures
import chess.pgn
import chess.engine
import random
import time
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm

In [2]:
def fen_to_vec(fen, w_elo, b_elo, w_time, b_time, inc, thres): 
    board = chess.Board(fen)
    n_moves = len(list(board.legal_moves))
    with chess.engine.SimpleEngine.popen_uci(r"C:\Users\chinc\Desktop\Project_Assistant\Chess_Code\stockfish\stockfish-windows-x86-64-sse41-popcnt") as engine:
        eval = str(engine.analyse(board, chess.engine.Limit(time = 0.1))['score'].white())
        vars = engine.analyse(board, chess.engine.Limit(time = 0.1), multipv = n_moves)
    if eval[0] == '#':
        eval = (eval[1] == '+')*5000 - (eval[1] == '-')*5000
    w_wins = draws = 0
    for var in vars:
        var_score = str(var['score'].white())
        if var_score[0] == '#':
            if var_score[1] == '+':
                w_wins += 1
        elif int(var_score) > thres:
            w_wins += 1
        elif -thres <= int(var_score) <= thres:
            draws += 1
    ply = board.fullmove_number*2 - board.turn
    vec = np.array([n_moves, board.turn, ply, w_elo, b_elo, w_time, b_time, inc, w_wins, draws, int(eval)])
    return vec

In [3]:
def eval_fun(fen): #returns the evaluation of all the variations
    evals = []
    moves = []
    board = chess.Board(fen)
    n_moves = len(list(board.legal_moves))
    with chess.engine.SimpleEngine.popen_uci(r"C:\Users\chinc\Desktop\Project_Assistant\Chess_Code\stockfish\stockfish-windows-x86-64-sse41-popcnt") as engine:
        vars = engine.analyse(board, chess.engine.Limit(time = 0.1), multipv = n_moves)
    for var in vars:
        var_score = str(var['score'].white())
        if var_score[0] == '#':
            evals.append((var_score[1] == '+')*5000 - (var_score[1] == '-')*5000)
        else:
            evals.append(int(var_score))
        moves.append(str(var['pv'][0]))  
    stock_policy = {move:eval for move, eval in zip(moves, evals)}
    return fen, stock_policy

READING THE DATA FROM THE FILE

In [4]:
def num_moves(fen):
    board = chess.Board(fen)
    return len(list(board.legal_moves))

In [5]:
#Reading data from the file
X = []
with open('game_details_win_percent2.txt', 'r') as f:
    for line in f:
        X.append(line.strip().split('$'))
X = np.array(X)
X = np.concatenate(((X[:,1].astype(int)%2).reshape(-1, 1), X), axis = 1)

In [6]:
X[0]

array(['0', 'rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R b - - 0 17',
       '34', '2059', '1941', '663.512', '399.17199999999997', '10',
       'e6f5', '1'], dtype='<U76')

In [8]:
#fens = X[:, 1]
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     n_moves = np.array(list(executor.map(num_moves, fens)))
# X = np.concatenate((n_moves.reshape(-1, 1), X), axis = 1)

In [7]:
#Number of legal moves for each piece
fens = X[:, 1]
def legal_moves_each_piece(fen):
    board = chess.Board(fen)
    
    legal_moves_count = {
        'P1':0, 'P2':0, 'P3':0, 'P4':0, 'P5':0, 'P6':0, 'P7':0, 'P8':0,  #White pawns
        'N1':0, 'N2':0,   #White knights
        'B1':0, 'B2':0,   #White Bishops
        'R1':0, 'R2':0,   #White Rooks
        'Q':0, 'K':0,     #White king and queen
        'p1':0, 'p2':0, 'p3':0, 'p4':0, 'p5':0, 'p6':0, 'p7':0, 'p8':0,  #Black pawns
        'n1':0, 'n2':0,   #Black knights
        'b1':0, 'b2':0,   #Black Bishops
        'r1':0, 'r2':0,   #Black Rooks
        'q':0, 'k':0,     #Black king and queen
    }
    
    piece_counts = {
        'P':0, 'N':0, 'B':0, 'R':0, 'Q':0, 'K':0,     #White pieces
        'p':0, 'n':0, 'b':0, 'r':0, 'q':0, 'k':0,     #Black pieces
    }
    
        # Function to generate piece names with counts
    def get_piece_name(piece, square):
        symbol = piece.symbol()
        piece_counts[symbol] += 1
        if symbol in 'Pp':
            # Separate pawns based on file (a-h) to distinguish them
            return symbol + str(chess.square_file(square) + 1)
        else:
            return symbol + str(piece_counts[symbol])

    # Iterate through all legal moves
    for move in board.legal_moves:
        # Get the piece at the origin square of the move
        piece = board.piece_at(move.from_square)
        if piece is not None:
            piece_name = get_piece_name(piece, move.from_square)
            if piece_name in legal_moves_count:
                legal_moves_count[piece_name] += 1

    # Create an array in the order of pieces
    piece_order = [
        'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8',
        'N1', 'N2', 'B1', 'B2', 'R1', 'R2', 'Q', 'K',
        'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8',
        'n1', 'n2', 'b1', 'b2', 'r1', 'r2', 'q', 'k'
    ]
    legal_moves_array = [legal_moves_count.get(piece, 0) for piece in piece_order]

    return legal_moves_array

legal_moves = []
for i in range(len(fens)):
    legal_moves.append(legal_moves_each_piece(fens[i]))
legal_moves = np.array(legal_moves)

In [8]:
X = np.concatenate((legal_moves, X),axis=1)
X.shape

(325348, 42)

In [9]:
X[0]

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '2', '2', '0', '1', '2', '1', '2', '2', '1', '1',
       '0', '0', '1', '1', '0', '0', '0',
       'rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R b - - 0 17', '34',
       '2059', '1941', '663.512', '399.17199999999997', '10', 'e6f5', '1'],
      dtype='<U76')

In [10]:
fens_in_file = []
with open('eval_features_win_percent_vars3.txt', 'r') as f:
    for line in f:
        fens_in_file.append(line.strip().split('$')[0])

In [11]:
fens_not_in_file = set(fens_in_file) - set(X[:, 0])
len(fens_not_in_file)

323334

In [12]:
#To find the ply number and side to play

def ply_number_fun(fen):
    board = chess.Board(fen)
    ply_number = board.fullmove_number * 2 - (1 if board.turn == chess.WHITE else 0)
    return ply_number

def side_to_play_fun(fen):
    board = chess.Board(fen)
    side_to_play = 1 if board.turn == chess.WHITE else 0
    return side_to_play

In [16]:
#len(df2)

In [ ]:
#fens = X[:, 2]
fens = fens_not_in_file
#253766
#eval_features = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    #eval_features = np.array(list(executor.map(aug, fens)))
    progress = tqdm(total = len(fens))
    futures = [executor.submit(eval_fun, fen) for fen in fens]
    for future in concurrent.futures.as_completed(futures):
        # Update progress bar
        result = future.result()
        with open('eval_features_win_percent_vars3.txt', 'a') as f:
            #f.write('$'.join([str(item) for item in result]) + '\n')
            f.write(result[0] + '$' + str(result[1]) + '\n')
        #eval_features.append(result)
        progress.update(1)
    progress.close()
#eval_features = np.array(eval_features)

In [13]:
df1 = pd.DataFrame(X[:, :-2])
df1.columns = ['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15', 'm16', 'm17', 'm18', 'm19', 'm20', 'm21', 'm22', 'm23', 'm24', 'm25', 'm26', 'm27', 'm28', 'm29', 'm30', 'm31', 'm32', 'side to play', 'fens', 'plies', 'w_elo', 'b_elo', 'w_time', 'b_time', 'inc_time']
df1['plies'] = df1['plies'].astype(int)
df1['w_elo'] = df1['w_elo'].astype(int)
df1['b_elo'] = df1['b_elo'].astype(int)
df1['w_time'] = df1['w_time'].astype(float)
df1['b_time'] = df1['b_time'].astype(float)
df1['inc_time'] = df1['inc_time'].astype(float)
df1['w_time'] = df1['w_time'] + df1['plies']//2*df1['inc_time']
df1['b_time'] = df1['b_time'] + (df1['plies'] - 1)//2*df1['inc_time']
df1.head(5)

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,m31,m32,side to play,fens,plies,w_elo,b_elo,w_time,b_time,inc_time
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R ...,34,2059,1941,833.512,559.172,10.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3r3k/1p3ppp/p1n1r3/3N1p2/1PP5/P2P2PP/3K2P1/4R2...,46,2059,1941,781.438,483.189,10.0
2,1,1,2,0,0,0,1,1,0,0,...,0,0,1,4k3/1p6/p5pp/3p1p2/1PP4P/P1K3P1/6P1/8 w - - 0 34,67,2059,1941,702.540,307.390,10.0
3,1,1,0,0,0,0,1,1,0,0,...,0,0,1,8/1p6/p2k2pp/3P1p2/PP5P/2K3P1/6P1/8 w - - 1 36,71,2059,1941,684.551,316.295,10.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,r2q1rk1/1pp1b1pp/2np4/p7/2Pp4/P4BPP/1P1BQP2/4R...,36,2743,2319,602.324,790.941,0.0


In [14]:
fens_eval = []
evals = []
with open('eval_features_win_percent_vars3.txt', 'r') as f:
    for line in f:
        res = line.strip().split('$')
        fens_eval.append(res[0])
        evals.append(res[1])

In [15]:
df2 = pd.DataFrame({'fens':fens_eval, 'stock_evals':evals})
df2 = df2.drop_duplicates(subset = ['fens'])
df2.head(5)

,fens,stock_evals
0,r4rk1/2p2ppp/p1b1p3/1p1n4/1P1PN2q/PQ1BP3/5PPK/...,{'h2g1': 248}
1,1k3b1r/1q5p/p3R3/4Q2P/1P6/P7/5PPK/2r5 b - - 1 32,"{'b7c7': -544, 'b8a8': -182, 'b8c8': -156, 'b8..."
2,r1bn2k1/ppp2pp1/5n1p/3PN3/q1P5/B2B4/P2Q1PPP/4R...,"{'a3e7': 468, 'a3c5': 408, 'e1e3': 393, 'd2c3'..."
3,4k3/1p6/p5pp/3p1p2/1PP4P/P1K3P1/6P1/8 w - - 0 34,"{'c4d5': 206, 'c3d4': 131, 'c3d3': 130, 'a3a4'..."
4,2b4r/6pp/pk3p2/1pb1P3/2p2P2/r1n2NP1/3N2BP/R3R2...,"{'a1c1': -406, 'd2c4': -418, 'a1a3': -444, 'e5..."


In [16]:
df_merged = df1.merge(df2, on = 'fens', how = 'left')
df_merged.head(5)

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,m32,side to play,fens,plies,w_elo,b_elo,w_time,b_time,inc_time,stock_evals
0,0,0,0,0,0,0,0,0,0,0,...,0,0,rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R ...,34,2059,1941,833.512,559.172,10.0,"{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6'..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,3r3k/1p3ppp/p1n1r3/3N1p2/1PP5/P2P2PP/3K2P1/4R2...,46,2059,1941,781.438,483.189,10.0,"{'g7g5': 232, 'h8g8': 253, 'b7b5': 254, 'g7g6'..."
2,1,1,2,0,0,0,1,1,0,0,...,0,1,4k3/1p6/p5pp/3p1p2/1PP4P/P1K3P1/6P1/8 w - - 0 34,67,2059,1941,702.540,307.390,10.0,"{'c4d5': 206, 'c3d4': 131, 'c3d3': 130, 'a3a4'..."
3,1,1,0,0,0,0,1,1,0,0,...,0,1,8/1p6/p2k2pp/3P1p2/PP5P/2K3P1/6P1/8 w - - 1 36,71,2059,1941,684.551,316.295,10.0,"{'c3c4': 284, 'h4h5': 283, 'c3d4': 229, 'g3g4'..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,r2q1rk1/1pp1b1pp/2np4/p7/2Pp4/P4BPP/1P1BQP2/4R...,36,2743,2319,602.324,790.941,0.0,"{'e7f6': 34, 'f8e8': 72, 'e7g5': 82, 'f8f3': 1..."


In [19]:
df_save = df_merged.iloc[:, :41]
df_save['result'] = df_merged['result']
df_save.head(5)

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,side to play,fens,plies,w_elo,b_elo,w_time,b_time,inc_time,stock_evals,result
0,0,0,0,0,0,0,0,0,0,0,...,0,rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R ...,34,2059,1941,833.512,559.172,10.0,"{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6'...",1
1,0,0,0,0,0,0,0,0,0,0,...,0,3r3k/1p3ppp/p1n1r3/3N1p2/1PP5/P2P2PP/3K2P1/4R2...,46,2059,1941,781.438,483.189,10.0,"{'g7g5': 232, 'h8g8': 253, 'b7b5': 254, 'g7g6'...",1
2,1,1,2,0,0,0,1,1,0,0,...,1,4k3/1p6/p5pp/3p1p2/1PP4P/P1K3P1/6P1/8 w - - 0 34,67,2059,1941,702.540,307.390,10.0,"{'c4d5': 206, 'c3d4': 131, 'c3d3': 130, 'a3a4'...",1
3,1,1,0,0,0,0,1,1,0,0,...,1,8/1p6/p2k2pp/3P1p2/PP5P/2K3P1/6P1/8 w - - 1 36,71,2059,1941,684.551,316.295,10.0,"{'c3c4': 284, 'h4h5': 283, 'c3d4': 229, 'g3g4'...",1
4,0,0,0,0,0,0,0,0,0,0,...,0,r2q1rk1/1pp1b1pp/2np4/p7/2Pp4/P4BPP/1P1BQP2/4R...,36,2743,2319,602.324,790.941,0.0,"{'e7f6': 34, 'f8e8': 72, 'e7g5': 82, 'f8f3': 1...",1


In [20]:
df_save.to_csv('game_data_final_win_loss.csv', index = False)

In [17]:
w_plus_all = []
b_plus_all = []
equals_all = []
pos_evals = []
thres = 50
stock_evals = df_merged['stock_evals'].values
for multi_eval in stock_evals:
    stock_policy = ast.literal_eval(multi_eval)
    pos_evals.append(list(stock_policy.values())[0])
    w_plus = b_plus = equals = 0
    for eval in stock_policy.values():
        if eval > thres:
            w_plus += 1
        elif eval < -thres:
            b_plus += 1
        else:
            equals += 1
    w_plus_all.append(w_plus)
    b_plus_all.append(b_plus)
    equals_all.append(equals)

In [18]:
df_merged['white_plus'] = w_plus_all
df_merged['black_plus'] = b_plus_all
df_merged['equals'] = equals_all
df_merged['pos_evals'] = pos_evals
df_merged['result'] = X[:, -1].astype(int)
df_merged.head(5)

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,b_elo,w_time,b_time,inc_time,stock_evals,white_plus,black_plus,equals,pos_evals,result
0,0,0,0,0,0,0,0,0,0,0,...,1941,833.512,559.172,10.0,"{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6'...",20,0,0,239,1
1,0,0,0,0,0,0,0,0,0,0,...,1941,781.438,483.189,10.0,"{'g7g5': 232, 'h8g8': 253, 'b7b5': 254, 'g7g6'...",37,0,0,232,1
2,1,1,2,0,0,0,1,1,0,0,...,1941,702.540,307.390,10.0,"{'c4d5': 206, 'c3d4': 131, 'c3d3': 130, 'a3a4'...",7,3,2,206,1
3,1,1,0,0,0,0,1,1,0,0,...,1941,684.551,316.295,10.0,"{'c3c4': 284, 'h4h5': 283, 'c3d4': 229, 'g3g4'...",3,5,3,284,1
4,0,0,0,0,0,0,0,0,0,0,...,2319,602.324,790.941,0.0,"{'e7f6': 34, 'f8e8': 72, 'e7g5': 82, 'f8f3': 1...",31,0,1,34,1


In [25]:
eval_features = []
thres = 80
with open('eval_features_win_percent_vars.txt', 'r') as f:
    for line in f:
        stock_policy = ast.literal_eval(line.strip())
        w_wins = draws = 0
        for eval in stock_policy.values():
            if eval > thres:
                w_wins += 1
            elif -thres <= eval <= thres:
                draws += 1
        eval_features.append([w_wins, draws])#, list(stock_policy.values())[0]])
eval_features = np.array(eval_features)
evals = []
with open('eval_features_win_percent.txt', 'r') as f:
    for line in f:
        evals.append(int(line.strip()))
eval_features = np.concatenate((eval_features, np.array(evals).reshape(-1, 1)), axis = 1)

FileNotFoundError: [Errno 2] No such file or directory: 'eval_features_win_percent_vars.txt'

In [ ]:
X[0]

In [ ]:
Y = X[:, -1].astype(int)
ohe = OneHotEncoder(sparse_output = False)
Y_ohe = ohe.fit_transform(Y.reshape(-1, 1))
print(ohe.categories_)

In [ ]:
X_final = df_merged.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]].values
X_final[0]

In [ ]:
# X = np.concatenate((np.array([[float(Xij) for Xij in Xi[[0, 1, 3, 4, 5, 6, 7, 8]]] for Xi in X]), eval_features.reshape(-1,1)), axis = 1)
X_final = np.concatenate((np.array([[float(Xij) for Xij in Xi[[0, 1, 3, 4, 5, 6, 7, 8]]] for Xi in X]), eval_features), axis = 1)

In [ ]:
X_final[1]

In [ ]:
x_train, x_test_val, y_train, y_test_val = train_test_split(X_final, Y_ohe, test_size = 0.3, shuffle = True, random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size = 0.6)

In [ ]:
evals = []
models = []
scores = []
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5)
with open('eval_features_win_percent.txt', 'r') as f:
    for line in f:
        evals.append(int(line.strip()))
for thres in range(50, 100, 10):
    eval_features = []
    with open('eval_features_win_percent_vars.txt', 'r') as f:
        for line in f:
            stock_policy = ast.literal_eval(line.strip())
            w_wins = draws = 0
            for eval in stock_policy.values():
                if eval > thres:
                    w_wins += 1
                elif -thres <= eval <= thres:
                    draws += 1
            eval_features.append([w_wins, draws])#, list(stock_policy.values())[0]])
    eval_features = np.concatenate((np.array(eval_features), np.array(evals).reshape(-1, 1)), axis = 1)
    X_final = np.concatenate((np.array([[float(Xij) for Xij in Xi[[0, 1, 3, 4, 5, 6, 7, 8]]] for Xi in X]), eval_features), axis = 1)
    x_train, x_test_val, y_train, y_test_val = train_test_split(X_final, Y_ohe, test_size = 0.3, shuffle = True, random_state = 0)
    x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size = 0.6)
    ni = len(X[0])
    no = len(Y_ohe[0])
    # best 192 2 layers, accuracy test = 73.22%
    model = Sequential([
        Dense(256, input_shape = (ni,), activation = 'relu'),
        #Dropout(0.1),
        Dense(256, activation = 'relu'),
        #Dropout(0.1),
        #Dense(192, activation = 'relu'),
        Dense(no, activation = 'softmax')
    ])
    model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
    )
    model.fit(x_train, y_train, epochs = 50, validation_data = (x_val, y_val), batch_size = 32, callbacks = [early_stopping_callback], verbose = 0)
    y_pred = model.predict(x_test)
    y_pred = np.argmax(y_pred, axis = 1) - 1
    y_true = ohe.inverse_transform(y_test).reshape(-1)
    score = accuracy_score(y_true, y_pred)*100
    print(f'The test score for threshold = {thres} is {score}%.')
    scores.append(score)
    models.append(model)
    del model
print(f'The best threshold = {50 + 10*np.argmax(scores)} with accuracy {max(scores)}%')

BUILDING THE MODEL WIN DRAW LOSS MODEL

In [ ]:
!pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier
x_train_clf, x_test_val_clf, y_train_clf, y_test_val_clf = train_test_split(X_final, Y, test_size = 0.3, shuffle = True, random_state = 0)
x_val_clf, x_test_clf, y_val_clf, y_test_clf = train_test_split(x_test_val_clf, y_test_val_clf, test_size = 0.6)
clf = LazyClassifier(verbose = 1, ignore_warnings = False, custom_metric = None)
models, predictions = clf.fit(x_train_clf, x_test_clf, y_train_clf, y_test_clf)
print(predictions)

In [ ]:
data = pd.read_csv('game_data_final_win_loss.csv')
y = data['result'].values
data_x = data.iloc[:, 1:-1]

In [ ]:
data_x.head(5)

In [ ]:
w_plus_all = []
b_plus_all = []
equals_all = []
pos_evals = []
thres = 50
stock_evals = data_x['stock_evals'].values
for multi_eval in stock_evals:
    stock_policy = ast.literal_eval(multi_eval)
    pos_evals.append(list(stock_policy.values())[0])
    w_plus = b_plus = equals = 0
    for eval in stock_policy.values():
        if eval > thres:
            w_plus += 1
        elif eval < -thres:
            b_plus += 1
        else:
            equals += 1
    w_plus_all.append(w_plus)
    b_plus_all.append(b_plus)
    equals_all.append(equals)

In [ ]:
data_x = data_x.iloc[:, :-1]
data_x['white_plus'] = w_plus_all
data_x['black_plus'] = b_plus_all
data_x['equals'] = equals_all
data_x['pos_evals'] = pos_evals
data_x.head(5)

In [ ]:
x = data_x.values
x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size = 0.3, random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size = 0.6)

In [ ]:
!pip install catboost

In [ ]:
import catboost
from catboost import CatBoostClassifier
model_cat = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, eval_metric='Accuracy', random_seed=42, logging_level='Verbose', use_best_model=True, early_stopping_rounds=100)
model_cat.fit(x_train, y_train, eval_set = (x_val, y_val))
y_pred_cat = model_cat.predict(x_test)
print(f'The accuracy of the catboost model is: {accuracy_score(y_test, y_pred_cat)}')

In [ ]:
ni = len(x_train[0])
no = len(Y_ohe[0])
alpha = 5
ns = len(x_train)
nh = int(ns/(ni + no)/alpha)
# best 192 2 layers, accuracy test = 73.22%
model = Sequential([
    Dense(192, input_shape = (ni,), activation = 'relu'),
    #Dropout(0.1),
    #Dense(400, activation = 'relu'),
    #Dropout(0.1),
    #Dense(400, activation = 'relu'),
    Dense(192, activation = 'relu'),
    Dense(no, activation = 'softmax')
])

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5)
model.fit(x_train, y_train, epochs = 50, validation_data = (x_val, y_val), batch_size = 32, callbacks = [early_stopping_callback])

MODEL EVALUATION

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis = 1) - 1
y_true = ohe.inverse_transform(y_test).reshape(-1)
print(accuracy_score(y_true, y_pred)*100)
confusion_matrix(y_true, y_pred)

COMPARISON WITH STOCKFISH EVALUATIONS ONLY MODEL

In [ ]:
bd = 50 #equality threshold
y_pred_stock = np.array([it > bd for it in x_test[:, -1]]).astype(int) - np.array([it < -bd for it in x_test[:, -1]]).astype(int)
y_true = ohe.inverse_transform(y_test).reshape(-1)
print(accuracy_score(y_true, y_pred_stock)*100)
confusion_matrix(y_true, y_pred_stock)

In [ ]:
model.save('best_win_loss_model_final2.h5')

RUNNING AN ENTRY

In [ ]:
model = tf.keras.models.load_model('best_win_loss_model_var.keras')

In [ ]:
samp_fen = 'r2r4/pq4bk/4Q1pp/1PP2p2/RB2p3/3p2NP/5PK1/3R4 b - - 3 29'
w_elo = 2647
b_elo = 2704
inc = 2
w_time = 12
b_time = 13
thres = 0.8
vec = fen_to_vec(samp_fen, w_elo, b_elo, w_time, b_time, inc, thres)
print(vec)
#model.predict(vec.reshape(-1, 1))

In [ ]:
b_samp = chess.Board(samp_fen)
display(b_samp)

In [ ]:
model.predict(np.array([vec]))